# Jaguar向量存储

本文档演示了llama_index与Jaguar向量存储一起使用的情况。

- 这是一个分布式向量数据库，可以存储大量向量。
- ZeroMove功能实现了即时的水平扩展。
- 它支持嵌入、文本、图像、视频、PDF、音频、时间序列和空间数据。
- 全主节点架构允许并行读写。
- 其异常检测功能可以区分数据集中的异常值。
- RAG支持可以结合LLMs和专有实时数据。
- 跨多个向量索引共享元数据可以提高数据一致性。
- 距离度量包括欧氏距离、余弦距离、内积、曼哈顿距离、切比雪夫距离、汉明距离、杰卡德距离和闵可夫斯基距离。
- 可以使用时间截止和时间衰减效应进行相似性搜索。

## 先决条件

在运行本文件中的示例之前，有两个要求。

您必须安装并设置JaguarDB服务器及其HTTP网关服务器。
请参考[Jaguar设置](http://www.jaguardb.com/docsetup.html)中的说明。

您必须安装llama-index和jaguardb-http-client包。

```bash
docker pull jaguardb/jaguardb_with_http
docker run -d -p 8888:8888 -p 8080:8080 --name jaguardb_with_http jaguardb/jaguardb_with_http
pip install -U llama-index
pip install -U jaguardb-http-client
```


In [ ]:
%pip install llama-index-vector-stores-jaguar

In [ ]:
!pip install -U jaguardb-http-client

  Using cached jaguardb_http_client-3.4.1-py2.py3-none-any.whl (15 kB)


## 导入
应该导入以下包。我们以OpenAIEmbedding为例。您可以在您的应用程序中选择其他嵌入模型。


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.vector_stores.jaguar import JaguarVectorStore
from jaguardb_http_client.JaguarHttpClient import JaguarHttpClient

## 客户端对象
现在我们实例化一个 jaguar 向量存储客户端对象。url 是网关服务器的 http 端点。url 应该根据您的环境设置进行替换。pod 是 Pod（或数据库）的名称。store 是向量存储的名称。一个 pod 可能有多个存储。vector_index 是存储中向量索引的名称。一个存储可能有多个向量索引。然而，存储客户端对象仅绑定到一个向量索引。vector_type 指定了向量索引的属性。在字符串 "cosine_fraction_short" 中，cosine 表示计算两个向量之间的距离时使用余弦距离。Fraction 表示向量分量是分数。Short 表示向量分量的存储格式是带符号的 16 位整数的短整数。存储格式可以是 32 位浮点数的浮点数。它也可以是 8 位带符号整数的字节。vector_dimension 是由提供的嵌入模型生成的向量的维度。


In [ ]:
url = "http://127.0.0.1:8080/fwww/"pod = "vdb"store = "llamaindex_jaguar_store"vector_index = "v"vector_type = "cosine_fraction_float"# vector_type = "cosine_fraction_short"  # 与float相比，内存使用量减半# vector_type = "cosine_fraction_byte"  # 与float相比，内存使用量减少四分之一vector_dimension = 1536  # 每个OpenAIEmbedding模型jaguarstore = JaguarVectorStore(    pod,    store,    vector_index,    vector_type,    vector_dimension,    url,)

## 认证
客户端必须登录或连接到后端jaguar服务器以确保系统安全和用户认证。环境变量JAGUAR_API_KEY或文件$HOME/.jagrc文件必须包含系统管理员颁发的jaguar api密钥。login()方法返回True或False。如果返回False，则可能意味着您的jaguar api密钥无效，或者http网关服务器未运行，或者jaguar服务器未正常运行。


In [ ]:
true_or_false = jaguarstore.login()
print(f"login result is {true_or_false}")

login result is True


## 创建向量存储
现在我们创建一个向量存储，其中包含一个大小为1024字节的字段 'v:text' 用于存储文本，以及两个额外的元数据字段 'author' 和 'category'。


In [ ]:
metadata_str = "author char(32), category char(16)"
text_size = 1024
jaguarstore.create(metadata_str, text_size)

## 加载文档
以下代码打开了示例Paul Gram文档，并将其读入内存中。


In [ ]:
documents = SimpleDirectoryReader("../data/paul_graham/").load_data()
print(f"loading {len(documents)} doument(s)")

loading 1 doument(s)


## 创建索引
准备存储上下文、服务上下文，并创建一个索引对象。在调用from_documents()之后，将会在向量存储中保存22个向量。


In [ ]:
### 使用我们的向量存储创建一个存储上下文storage_context = StorageContext.from_defaults(vector_store=jaguarstore)### 清除向量存储中的所有向量jaguarstore.clear()### 使用文档和存储上下文创建索引index = VectorStoreIndex.from_documents(    documents, storage_context=storage_context)### 您可以向向量存储中添加更多文档：# jaguarstore.add_documents(some_docs)# jaguarstore.add_documents(more_docs, text_tag="对这些文档进行标记")### 打印jaguar向量存储中的文档数量num = jaguarstore.count()print(f"jaguar向量存储中有 {num} 个向量")

There are 22 vectors in jaguar vector store


## 提出问题
我们使用一个查询引擎并向引擎提出一些问题。


In [ ]:
query_engine = index.as_query_engine()
q = "What did the author do growing up?"
print(f"Question: {q}")
response = query_engine.query(q)
print(f"Answer: {str(response)}\n")

q = "What did the author do after his time at Viaweb?"
print(f"Question: {q}")
response = query_engine.query(q)
print(f"Answer: {str(response)}")

Question: What did the author do growing up?
Answer: The author mentioned that growing up, they worked on two main things outside of school: writing and programming. They wrote short stories and tried writing programs on an IBM 1401 computer.

Question: What did the author do after his time at Viaweb?
Answer: After his time at Viaweb, the author started a company to put art galleries online. However, this idea did not turn out to be successful as art galleries did not want to be online.


## 传递查询选项
我们可以通过使用`vector_store_kwargs`参数向查询引擎传递额外的参数，以仅从jaguar向量存储中选择数据的子集。参数`day_cutoff`是超出该天数的文本将被忽略的天数。`day_decay_rate`是相似度分数的每日衰减率。


In [ ]:
qkwargs = {
    "args": "day_cutoff=365,day_decay_rate=0.01",
    "where": "category='startup' or category=''",
}
query_engine_filter = index.as_query_engine(vector_store_kwargs=qkwargs)
q = "What was the author's life style?"
print(f"Question: {q}")
response = query_engine_filter.query(q)
print(f"Answer: {str(response)}")

Question: What was the author's life style?
Answer: The author's lifestyle involved attending the Accademia as a student and painting still lives in their bedroom at night. They also wrote essays and had a messy life, which they thought would be interesting and encouraging to others.


## 清理和注销
可以删除向量存储中的所有向量和相关数据，并完全移除向量存储以完成测试。注销调用确保客户端使用的资源被释放。


In [ ]:
### 如果需要，清空向量存储中的所有数据jaguarstore.clear()### 如果需要，从数据库中删除整个向量jaguarstore.drop()### 与jaguar服务器断开连接并清理资源jaguarstore.logout()